<div style="text-align:center">
<img src="img/nlopt2.gif" width="600">
</div>

# <b> Summary </b>
1. <b> French Gemini AI : Répartition de la charge électrique (Economic Dispatch) - Other language test</b>
    - Abstract
    - Instance
    - Math model
    - Cplex solution
2. <b> ChatGpt AI : Factory Electricity load balancing</b>
    - Abstract
    - Instance
    - Math model
    - Cplex solution
3. <b> DeepSeek A.I : Non linear facility location</b>
    - Abstract
    - Instance
    - Math model
    - Cplex solution
4. <b> ChatGpt A.I : Non linear facility location</b>
    - Abstract
    - Instance
    - Math model
    - Cplex solution


# <b> 1. Gemini A.I : Répartition de la charge électrique (Economic Dispatch)</b>



<div style="text-align:center">
<img src="img/top-10-nuclear-power-companies.png" width="400">
</div>

<div style="text-align:center">
<img src="img/gemini.png" width="200"/>
</div>

## <b>Abstract</b>
L'optimisation quadratique (QP) est extrêmement courante dans le monde réel, notamment pour équilibrer les coûts et la performance. Un exemple parfait est la répartition de la charge électrique (Economic Dispatch).

Imaginons que vous dirigiez une petite centrale électrique avec deux générateurs. Votre but est de produire une quantité précise d'électricité au coût le plus bas possible.

Le Problème : Pourquoi "Quadratique" ?

Le coût de fonctionnement d'un générateur n'est pas linéaire. Plus on le sollicite, plus il perd en efficacité (chaleur, usure), ce qui fait grimper le coût de façon exponentielle. On modélise souvent cela par une fonction du second degré :

Cout=ax²+bx+c

## <b>Instance</b>


<div style="text-align:center">
<img src="img/Screenshot 2026-01-22 at 21-30-55 Google Gemini.png" width="400">
</div>

## <b>Math model</b>


<div style="text-align:center">
<img src="img/Screenshot 2026-01-22 at 21-40-22 Google Gemini.png">
</div>

<div style="text-align:center">
<img src="img/Screenshot 2026-01-22 at 21-44-38 Google Gemini.png">
</div>

<div style="text-align:center">
<img src="img/Screenshot 2026-01-22 at 21-45-39 Google Gemini.png">
</div>






## <b>Cplex solution</b>
<div style="text-align:center">
<img src="img/IBM_CPLEX_0.png">
</div>

In [54]:
from docplex.mp.model import Model

# 1. Créer le modèle
mdl = Model(name='Optimisation_Energie')

# 2. Variables de décision (Puissance produite par chaque générateur en MW)
# On définit des bornes (ex: min 10MW, max 200MW)
x1 = mdl.continuous_var(lb=10, ub=200, name='gen1')
x2 = mdl.continuous_var(lb=10, ub=200, name='gen2')

# 3. Fonction Objectif : Minimiser le coût total (Quadratique)
# Coût = (0.5*x1^2 + 2*x1) + (0.25*x2^2 + 8*x2)
mdl.minimize(0.5*x1**2 + 2*x1 + 0.25*x2**2 + 8*x2)

# 4. Contrainte : Satisfaire la demande de 150 MW
mdl.add_constraint(x1 + x2 == 150, "Demande_Totale")

# 5. Résolution
solution = mdl.solve()

# 6. Affichage des résultats
if solution:
    print(f"Statut : {mdl.get_solve_status()}")
    print(f"Puissance Générateur 1 : {solution[x1]:.2f} MW")
    print(f"Puissance Générateur 2 : {solution[x2]:.2f} MW")
    print(f"Coût Total : {solution.objective_value:.2f} €")
else:
    print("Aucune solution trouvée.")

Statut : JobSolveStatus.OPTIMAL_SOLUTION
Puissance Générateur 1 : 54.00 MW
Puissance Générateur 2 : 96.00 MW
Coût Total : 4638.00 €


Pourquoi c'est intelligent ?

Si le coût était linéaire, CPLEX choisirait simplement le générateur le moins cher et le ferait tourner au maximum. Ici, comme le coût est quadratique, CPLEX calcule le point d'équilibre où l'augmentation du coût marginal du premier générateur devient plus élevée que celle du deuxième.

Dans cet exemple, CPLEX va répartir la charge de manière à ce que les "pentes" des coûts soient égales, minimisant ainsi la facture totale.

Ce que CPLEX apporte de plus

Dans un vrai scénario industriel, CPLEX gérerait en plus :

    Les pertes en ligne (la résistance des câbles).

    Les émissions de CO2 (une autre fonction quadratique à minimiser).

    Le démarrage/arrêt (variables binaires, ce qui transforme le problème en MIQP - Mixed Integer Quadratic Programming).

# <b> 2. ChatGpt AI : Factory Electricity load balancing</b>

<div style="text-align:center">
<img src="img/GalesbergSubstation.jpg" width="400" height="400">
</div>

<div style="text-align:center">
<img src="img/chatgpt.png" width="200"/>
</div>


## Abstract

<i>" Can you do a real life simple non linear example of Electric load balancing in a factory, using cplex ?"</i>

Yep — let’s do a real-life, simple, non-linear electric load balancing example that actually makes sense on a factory floor, and then show how you’d model it in CPLEX (MIQP / QP).

I’ll keep it concrete and practical, not textbook-fluffy.

## Instance : Real-life factory story (simple + realistic)
Imagine a factory with 3 machines that can be run during a given hour:

| Machine | Description         | Power (kW) | Notes                    |
| ------- | ------------------- | ---------- | ------------------------ |
| M1      | Cutting machine     | up to 50   | High efficiency          |
| M2      | Welding machine     | up to 40   | Medium efficiency        |
| M3      | Heat treatment oven | up to 60   | Inefficient at high load |

Reality constraints

- The factory must process 100 kW total load in this hour.

- Each machine has a maximum capacity.

- Electric losses and wear increase non-linearly when a machine is heavily loaded.

- Electricity tariff penalizes peaks → quadratic cost.

👉 This is why it’s non-linear.

## Math model

2️⃣ Decision variables

Let:

$$
x1​,x2​,x3 ​= power\thinspace  assigned\thinspace  to\thinspace  machines\thinspace  M1, M2, M3 (kW)
$$

3️⃣ Non-linear cost (very realistic)

Each machine has a quadratic operating cost:
$$
Cost_{i}​=a_{i}​x_{i}^2​+b_{i}x_{i}​
$$

Why quadratic?

- Higher current → higher losses

- Thermal stress

- Maintenance degradation

Exemple coefficients:

| Machine | (a_i) (loss factor) | (b_i) (energy price) |
| ------- | ------------------- | -------------------- |
| M1      | 0.01                | 5                    |
| M2      | 0.015               | 4                    |
| M3      | 0.03                | 3                    |

👉 M3 looks cheap per kW, but becomes very expensive when heavily loaded.

4️⃣ Optimization model (math)

<div style="text-align:center">
<img src="img/Screenshot 2026-01-22 at 22-34-22 Electric Load Balancing CPLEX.png">
</div>




## Cplex solution

<div style="text-align:center">
<img src="img/IBM_CPLEX_0.png">
</div>

In [55]:
from docplex.mp.model import Model

# Create model
mdl = Model(name='Factory_Load_Balancing')

# Decision variables
x1 = mdl.continuous_var(lb=0, ub=50, name='M1')
x2 = mdl.continuous_var(lb=0, ub=40, name='M2')
x3 = mdl.continuous_var(lb=0, ub=60, name='M3')

# Objective function (quadratic)
mdl.minimize(
    0.01 * x1**2 + 5 * x1 +
    0.015 * x2**2 + 4 * x2 +
    0.03 * x3**2 + 3 * x3
)

# Demand constraint
mdl.add_constraint(x1 + x2 + x3 == 100)

# Solve
solution = mdl.solve()

# Print solution
if solution:
    print("Optimal Load Allocation:")
    print(f"M1: {x1.solution_value:.2f} kW")
    print(f"M2: {x2.solution_value:.2f} kW")
    print(f"M3: {x3.solution_value:.2f} kW")
    print(f"Total Cost: {solution.objective_value:.2f}")
else:
    print("No solution found")

Optimal Load Allocation:
M1: 20.00 kW
M2: 40.00 kW
M3: 40.00 kW
Total Cost: 456.00


6️⃣ What kind of solution you’ll see (intuition)

CPLEX will:

- Load M1 and M2 more evenly

- Avoid pushing M3 too hard despite cheaper base price

- Smooth the load → no peaks

7️⃣ Why this is a non-linear example (important)

| Feature                 | Linear   | This model          |
| ----------------------- | -------- | ------------------- |
| Cost per kW             | Constant | Increases with load |
| Peak penalties          | ❌        | ✅                   |
| Loss modeling           | ❌        | ✅                   |
| Realistic for factories | ❌        | ✅                   |

8️⃣ Want to level this up?

I can easily extend this to:

🔌 Time-of-use tariffs

⚡ Transformer capacity limits

🏭 On/Off machines (binary + MIQP)

🔋 Battery + renewable integration

📉 Demand response incentives

Just tell me how real you want it 😄

#  <b>3. DeepSeek A.I : Non linear facility location </b>

<div style="text-align:center">
<img src="img/floc.jpg" width="400" height="400">
</div>

<div style="text-align:center">
<img src="img/deepseek.png" width="200"/>
</div>


<i>Can you explain a real life example of Non linear facility location with cplex ? </i>

Here's a comprehensive real-life example of a Non-linear Facility Location Problem solved with CPLEX, including mathematical formulation and implementation.


## <b> Abstract </b>

Real-Life Example: Oil Refinery Location Problem
Problem Description

An oil company needs to decide where to build new refineries to serve gasoline stations across a region. The non-linearity comes from:

    Economies of scale - Refinery costs decrease per unit as capacity increases (concave cost function)

    Transportation costs that increase non-linearly with distance (due to pipeline friction losses)

    Environmental impact that grows quadratically with pollution concentration


## <b> Math model </b>



# <b> Cplex solution</b>

In [ ]:
import cplex
from docplex.mp.model import Model
import numpy as np

class NonLinearRefineryLocation:
    def __init__(self, n_locations, n_stations):
        self.n_locations = n_locations
        self.n_stations = n_stations
        self.model = Model(name='Nonlinear_Refinery_Location')
        
    def create_data(self):
        """Generate sample data"""
        np.random.seed(42)
        
        self.demand = np.random.randint(100, 500, self.n_stations)
        self.fixed_costs = np.random.randint(500000, 2000000, self.n_locations)
        self.capacities = np.random.randint(2000, 10000, self.n_locations)
        
        # Distance matrix with random distances
        self.distances = np.random.uniform(10, 500, 
                                          (self.n_locations, self.n_stations))
        
        # Non-linear cost coefficients
        self.alpha = np.random.uniform(50, 150, self.n_locations)  # Production
        self.beta = 0.15  # Transportation
        self.gamma = 0.001  # Environmental
        
    def build_model(self):
        """Build the non-linear model"""
        # Decision variables
        x = self.model.binary_var_list(self.n_locations, name='x')
        y = self.model.continuous_var_matrix(
            self.n_locations, self.n_stations, 
            lb=0, name='y'
        )
        z = self.model.continuous_var_list(self.n_locations, lb=0, name='z')
        
        # Objective function - includes non-linear terms
        total_cost = 0
        
        # 1. Fixed costs (linear)
        total_cost += sum(self.fixed_costs[i] * x[i] 
                         for i in range(self.n_locations))
        
        # 2. Non-linear production costs using piecewise approximation
        # CPLEX can handle certain non-linearities directly or via piecewise linear
        for i in range(self.n_locations):
            # Add piecewise linear approximation for concave cost z^0.8
            breakpoints = [0, 1000, 3000, 6000, 10000]
            slopes = [self.alpha[i] * (b**0.8 - (b-1000)**0.8)/1000 
                     for b in breakpoints[1:]]
            
            # Add as piecewise linear constraint
            self.model.add_pwl(
                z[i],
                [self.alpha[i] * (b**0.8) for b in breakpoints],
                breakpoints,
                'production_cost_'+str(i)
            )
        
        # 3. Non-linear transportation costs (using CPLEX's built-in power function)
        for i in range(self.n_locations):
            for j in range(self.n_stations):
                transport_cost = self.beta * self.distances[i][j] * \
                               self.model.power(y[(i, j)], 1.2)
                total_cost += transport_cost
        
        # 4. Quadratic environmental costs
        for i in range(self.n_locations):
            env_cost = self.gamma * self.model.square(z[i])
            total_cost += env_cost
        
        # Set objective
        self.model.minimize(total_cost)
        
        # Constraints
        # Demand satisfaction
        for j in range(self.n_stations):
            self.model.add_constraint(
                sum(y[(i, j)] for i in range(self.n_locations)) == 
                self.demand[j],
                ctname=f'demand_{j}'
            )
        
        # Capacity and linking constraints
        for i in range(self.n_locations):
            self.model.add_constraint(
                sum(y[(i, j)] for j in range(self.n_stations)) <= 
                self.capacities[i] * x[i],
                ctname=f'capacity_{i}'
            )
            
            # Capacity utilization definition
            self.model.add_constraint(
                z[i] == sum(y[(i, j)] for j in range(self.n_stations)),
                ctname=f'utilization_{i}'
            )
    
    def solve(self):
        """Solve the model with CPLEX"""
        # Set CPLEX parameters for non-linear problems
        self.model.parameters.optimalitytarget = 3  # Global optimality for non-convex
        self.model.parameters.mip.strategy.file = 2  # Node file
        self.model.parameters.threads = 4
        
        print("Solving non-linear facility location problem...")
        solution = self.model.solve(log_output=True)
        
        if solution:
            print(f"Optimal cost: ${solution.objective_value:,.2f}")
            self.print_solution(solution)
        else:
            print("No solution found")
        
        return solution
    
    def print_solution(self, solution):
        """Print the solution"""
        print("\n=== SOLUTION ===")
        print("Refineries to open:")
        for i in range(self.n_locations):
            if solution.get_value(f'x_{i}') > 0.5:
                util = solution.get_value(f'z_{i}')
                print(f"  Location {i}: Utilization = {util:,.0f} units")
        
        print("\nTransportation plan (sample):")
        for i in range(min(3, self.n_locations)):
            for j in range(min(5, self.n_stations)):
                flow = solution.get_value(f'y_{i}_{j}')
                if flow > 0:
                    print(f"  Refinery {i} -> Station {j}: {flow:,.1f} units")

# Usage
def main():
    # Create and solve the problem
    problem = NonLinearRefineryLocation(n_locations=5, n_stations=20)
    problem.create_data()
    problem.build_model()
    solution = problem.solve()
    
    # Alternative: Using CPLEX's built-in non-linear capabilities
    solve_with_cplex_nlp()

def solve_with_cplex_nlp():
    """Alternative approach using CPLEX's native NLP interface"""
    import cplex
    
    cpx = cplex.Cplex()
    cpx.set_results_stream(None)
    
    # Define variables
    # CPLEX can handle quadratic objectives and constraints
    # For more complex non-linearities, use IloCplex::PiecewiseLinearFunction
    
    # Add variables
    names = ['x1', 'x2']
    cpx.variables.add(names=names, lb=[0, 0], ub=[100, 100])
    
    # Add quadratic objective (example: minimize x1^2 + x2^2)
    qmat = [[[0, 1], [2.0, 0.0]],  # x1^2 coefficient
            [[], []]]              # x2^2 coefficient
    cpx.objective.set_quadratic(qmat)
    cpx.objective.set_sense(cpx.objective.sense.minimize)
    
    # Solve
    cpx.solve()
    print(f"\nNLP Solution status: {cpx.solution.get_status_string()}")
    print(f"Objective value: {cpx.solution.get_objective_value()}")

if __name__ == "__main__":
    main()

AttributeError: 'Model' object has no attribute 'pwl'